#Paper: Model Explorations in Abstractive Summarisation

GROUP 15:  
Youning Xia (19054254)
Yan Song (9898418)
Shuyi Han (19060915)
Wan Jing Song (17130843)


This paper sets out to assess the performanceof Deep Reinforcement Learning (DRL) basedabstractive summarization models.  4 differentmodel variants are applied on 3 datasets andevaluate on ROUGE and BERTScores. Work-ing  on  the  novel  WikiHow  dataset  (Koupaeeand Wang, 2018) which is slightly more com-plex to train on has magnified the characteris-tics of the models.   It exposes the instabilityof  training  on  ROUGE-L  scores  and  suggestBERTScore as an alternative.

This notebook is to demonstrate the reproducibility of the testing result on Gigaword, CNN-DM and Wikihow dataset. However, given the 5-mins limits, it is recommended to run on **Gigaword dataset** only. The results on other two dataset have already been printed out.

#CODE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##Package

In [2]:
pip install rouge

In [3]:
pip install bert_score

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 552kB 7.0MB/s 
     |████████████████████████████████| 870kB 20.7MB/s 
     |████████████████████████████████| 1.0MB 47.2MB/s 
     |████████████████████████████████| 3.7MB 38.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=db56a33ca61e5ac85709f778fe3d8d07f8cc23c0d72694cbc07edfa371fe4313
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import glob
import random
import struct
import csv
from tensorflow.core.example import example_pb2
import numpy as np

import queue as Queue
import time
from random import shuffle
from threading import Thread


import tensorflow as tf

import torch as T
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F

import os

from rouge import Rouge
import argparse


import shutil
import collections
import tqdm

import gc
import bert_score
import argparse

import pandas as pd


##Vocab

In [0]:

SENTENCE_START = '<s>'      # <s> and </s> are used in the data files to segment the abstracts into sentences. They don't receive vocab ids.
SENTENCE_END = '</s>'       # or <t>, </t>

PAD_TOKEN = '[PAD]'         # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNKNOWN_TOKEN = '[UNK]'     # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]'  # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]'    # This has a vocab id, which is used at the end of untruncated target sequences

# Note: none of <s>, </s>, [PAD], [UNK], [START], [STOP] should appear in the vocab file.

class Vocab(object):
  """Vocabulary class for mapping between words and ids (integers)"""

  def __init__(self, vocab_file, max_size):
    """Creates a vocab of up to max_size words, reading from the vocab_file. If max_size is 0, reads the entire vocab file.

    param  vocab_file: path to the vocab file, which is assumed to contain "<word> <frequency>" on each line, sorted with most frequent word first. This code doesn't actually use the frequencies, though.
    param  max_size: integer. The maximum size of the resulting Vocabulary."""
    self._word_to_id = {}                                                       #dictionary with word as key
    self._id_to_word = {}                                                       #           with id as key
    self._count = 0                                                             # keeps track of total number of words in the Vocab

    # [UNK], [PAD], [START] and [STOP] get the ids 0,1,2,3.
    for w in [UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
      self._word_to_id[w] = self._count                                         #fill in the dict
      self._id_to_word[self._count] = w     
      self._count += 1

    # Read the vocab file and add words up to max_size
    with open(vocab_file, 'r') as vocab_f:
      for line in vocab_f:
        pieces = line.split()                                                   #list with two elements, the first is word the other one is its id
        if len(pieces) != 2:
          print('Warning: incorrectly formatted line in vocabulary file: %s\n' % line)
          continue
        w = pieces[0]                                                           #word
        if w in [SENTENCE_START, SENTENCE_END, UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:        #raise error if these words exist in vocab
          raise Exception('<s>, </s>, [UNK], [PAD], [START] and [STOP] shouldn\'t be in the vocab file, but %s is' % w)
        if w in self._word_to_id:                                             
          raise Exception('Duplicated word in vocabulary file: %s' % w)         #Duplication error
        self._word_to_id[w] = self._count                                       #assign id to word
        self._id_to_word[self._count] = w                                       #assign word to id
        self._count += 1                                                        #id increment
        if max_size != 0 and self._count >= max_size:                           #stop assigning when max_size has been reached
          print("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (max_size, self._count))
          break

    print("Finished constructing vocabulary of %i total words. Last word added: %s" % (self._count, self._id_to_word[self._count-1]))

  def word2id(self, word):
    """Returns the id (integer) of a word (string). Returns [UNK] id if word is OOV."""
    if word not in self._word_to_id:
      return self._word_to_id[UNKNOWN_TOKEN]
    return self._word_to_id[word]

  def id2word(self, word_id):
    """Returns the word (string) corresponding to an id (integer)."""
    if word_id not in self._id_to_word:
      raise ValueError('Id not found in vocab: %d' % word_id)
    return self._id_to_word[word_id]

  def size(self):
    """Returns the total size of the vocabulary"""
    return self._count





def example_generator(data_path, single_pass):
  """Generates tf.Examples from data files.
    Binary data format: <length><blob>. <length> represents the byte size
    of <blob>. <blob> is serialized tf.Example proto. The tf.Example contains
    the tokenized article text and summary.
    https://www.tensorflow.org/tutorials/load_data/tfrecord

    param  data_path:
      Path to tf.Example data files. Can include wildcards, e.g. if you have several training data chunk files train_001.bin, train_002.bin, etc, then pass data_path=train_* to access them all.
    param  single_pass:
      Boolean. If True, go through the dataset exactly once, generating examples in the order they appear, then return. Otherwise, generate random examples indefinitely.
    Yields:
      Deserialized tf.Example.
  """
  while True:
    filelist = glob.glob(data_path)                                             # get the list of datafiles
    assert filelist, ('Error: Empty filelist at %s' % data_path)                # check filelist isn't empty
    if single_pass:
      filelist = sorted(filelist)
    else:
      random.shuffle(filelist)                                                  #random shuffle the data if not single_pass
    for f in filelist:
      reader = open(f, 'rb')
      while True:
        len_bytes = reader.read(8)
        if not len_bytes: break # finished reading this file
        str_len = struct.unpack('q', len_bytes)[0]
        example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
        yield example_pb2.Example.FromString(example_str)
    if single_pass:
      print("example_generator completed reading all datafiles. No more data.")
      break


def article2ids(article_words, vocab):
  """Map the article words to their ids. Also return a list of OOVs in the article.
  Args:
    article_words: list of words (strings)
    vocab: Vocabulary object
  Returns:
    ids:
      A list of word ids (integers); OOVs are represented by their temporary article OOV number. If the vocabulary size is 50k and the article has 3 OOVs, then these temporary OOV numbers will be 50000, 50001, 50002.
    oovs:
      A list of the OOV words in the article (strings), in the order corresponding to their temporary article OOV numbers."""
  ids = []
  oovs = []
  unk_id = vocab.word2id(UNKNOWN_TOKEN)
  for w in article_words:
    i = vocab.word2id(w)
    if i == unk_id:                         # If w is OOV
      if w not in oovs:                     # Add to list of OOVs
        oovs.append(w)
      oov_num = oovs.index(w)               # This is 0 for the first article OOV, 1 for the second article OOV...
      ids.append(vocab.size() + oov_num)    # This is e.g. 50000 for the first article OOV, 50001 for the second...(append at the end)
    else:
      ids.append(i)
  return ids, oovs


def abstract2ids(abstract_words, vocab, article_oovs):
  """Map the abstract words to their ids. In-article OOVs are mapped to their temporary OOV numbers.
  Args:
    abstract_words: list of words (strings)
    vocab: Vocabulary object
    article_oovs: list of in-article OOV words (strings), in the order corresponding to their temporary article OOV numbers
  Returns:
    ids: List of ids (integers). In-article OOV words are mapped to their temporary OOV numbers. Out-of-article OOV words are mapped to the UNK token id."""
  ids = []
  unk_id = vocab.word2id(UNKNOWN_TOKEN)
  for w in abstract_words:
    i = vocab.word2id(w)
    if i == unk_id:                         # If w is an OOV word
      if w in article_oovs:                 # If w is an in-article OOV
        vocab_idx = vocab.size() + article_oovs.index(w)    # Map to its temporary article OOV number
        ids.append(vocab_idx)
      else:                                 # If w is an out-of-article OOV
        ids.append(unk_id)                  # Map to the UNK token id
    else:
      ids.append(i)
  return ids


def outputids2words(id_list, vocab, article_oovs):
  """Maps output ids to words, including mapping in-article OOVs from their temporary ids to the original OOV string (applicable in pointer-generator mode).

    param  id_list: list of ids (integers)
            vocab: Vocabulary object
    param  article_oovs: list of OOV words (strings) in the order corresponding to their temporary article OOV ids (that have been assigned in pointer-generator mode), or None (in baseline mode)
    Returns:
            words: list of words (strings)
  """
  words = []
  for i in id_list:
    try:
      w = vocab.id2word(i) # might be [UNK]
    except ValueError as e: # w is OOV
      assert article_oovs is not None, "Error: model produced a word ID that isn't in the vocabulary. This should not happen in baseline (no pointer-generator) mode"
      article_oov_idx = i - vocab.size()
      try:
        w = article_oovs[article_oov_idx]
      except ValueError as e: # i doesn't correspond to an article oov
        raise ValueError('Error: model produced word ID %i which corresponds to article OOV %i but this example only has %i article OOVs' % (i, article_oov_idx, len(article_oovs)))
    words.append(w) 
  return words


def abstract2sents(abstract):
  """Splits abstract text from datafile into list of sentences.
  Args:
    abstract: string containing <s> and </s> tags for starts and ends of sentences
  Returns:
    sents: List of sentence strings (no tags)"""
  cur = 0
  sents = []
  while True:
    try:
      start_p = abstract.index(SENTENCE_START, cur)
      end_p = abstract.index(SENTENCE_END, start_p + 1)
      cur = end_p + len(SENTENCE_END)
      sents.append(abstract[start_p+len(SENTENCE_START):end_p])
    except ValueError as e: # no more sentences
      return sents


## Batcher

In [0]:
max_batch_queue = 1000 

In [0]:

import queue as Queue
import time
from random import shuffle
from threading import Thread

import numpy as np
import tensorflow as tf


import random
random.seed(1234)

class Example(object):
  '''
  read article, abstrction and vocab and process them into batching-ready format
  '''

  def __init__(self, article, abstract_sentences, vocab):
    # Get ids of special tokens
    start_decoding = vocab.word2id(START_DECODING)      #data
    stop_decoding = vocab.word2id(STOP_DECODING)        #data

    # Process the article
    article_words = article.split()
    if len(article_words) > max_enc_steps:              #truncate the input article to max_enc_step length
      article_words = article_words[ : max_enc_steps]   
    self.enc_len = len(article_words)                   # store the length after truncation but before padding
    self.enc_input = [vocab.word2id(w) for w in article_words] # list of word ids; OOVs are represented by the id for UNK token

    # Process the abstract
    abstract = ' '.join(abstract_sentences)                 # string
    abstract_words = abstract.split()                       # list of strings, split by space
    abs_ids = [vocab.word2id(w) for w in abstract_words]    # list of word ids; OOVs are represented by the id for UNK token

    # Get the decoder input sequence and target sequence
    self.dec_input, _ = self.get_dec_inp_targ_seqs(abs_ids, max_dec_steps, start_decoding, stop_decoding)
    self.dec_len = len(self.dec_input)

    # If using pointer-generator mode, we need to store some extra info
    # Store a version of the enc_input where in-article OOVs are represented by their temporary OOV id; also store the in-article OOVs words themselves
    self.enc_input_extend_vocab, self.article_oovs = article2ids(article_words, vocab)     #data

    # Get a verison of the reference summary where in-article OOVs are represented by their temporary article OOV id
    abs_ids_extend_vocab = abstract2ids(abstract_words, vocab, self.article_oovs)           #data

    # Get decoder target sequence
    _, self.target = self.get_dec_inp_targ_seqs(abs_ids_extend_vocab, max_dec_steps, start_decoding, stop_decoding)

    # Store the original strings
    self.original_article = article
    self.original_abstract = abstract
    self.original_abstract_sents = abstract_sentences



  def get_dec_inp_targ_seqs(self, sequence, max_len, start_id, stop_id):
    inp = [start_id] + sequence[:]
    target = sequence[:]
    if len(inp) > max_len: # truncate
      inp = inp[:max_len]
      target = target[:max_len] # no end_token
    else: # no truncation
      target.append(stop_id) # end token
    assert len(inp) == len(target)
    return inp, target


  def pad_decoder_inp_targ(self, max_len, pad_id):              #padding
    while len(self.dec_input) < max_len:
      self.dec_input.append(pad_id)
    while len(self.target) < max_len:
      self.target.append(pad_id)


  def pad_encoder_input(self, max_len, pad_id):                 #padding
    while len(self.enc_input) < max_len:
      self.enc_input.append(pad_id)
    while len(self.enc_input_extend_vocab) < max_len:
      self.enc_input_extend_vocab.append(pad_id)



#-------------------------------------------------------------------------------------------------


class Batch(object):
  def __init__(self, example_list, vocab, batch_size):
    self.batch_size = batch_size
    self.pad_id = vocab.word2id(PAD_TOKEN)      # id of the PAD token used to pad sequences
    self.init_encoder_seq(example_list)         # initialize the input to the encoder
    self.init_decoder_seq(example_list)         # initialize the input and targets for the decoder
    self.store_orig_strings(example_list)       # store the original strings


  def init_encoder_seq(self, example_list):
    # Determine the maximum length of the encoder input sequence in this batch
    max_enc_seq_len = max([ex.enc_len for ex in example_list])

    # Pad the encoder input sequences up to the length of the longest sequence
    for ex in example_list:
      ex.pad_encoder_input(max_enc_seq_len, self.pad_id)

    # Initialize the numpy arrays
    # Note: our enc_batch can have different length (second dimension) for each batch because we use dynamic_rnn for the encoder.
    self.enc_batch = np.zeros((self.batch_size, max_enc_seq_len), dtype=np.int32)
    self.enc_lens = np.zeros((self.batch_size), dtype=np.int32)
    self.enc_padding_mask = np.zeros((self.batch_size, max_enc_seq_len), dtype=np.float32)

    # Fill in the numpy arrays
    for i, ex in enumerate(example_list):
      self.enc_batch[i, :] = ex.enc_input[:]
      self.enc_lens[i] = ex.enc_len
      for j in range(ex.enc_len):
        self.enc_padding_mask[i][j] = 1

    # For pointer-generator mode, need to store some extra info
    # Determine the max number of in-article OOVs in this batch
    self.max_art_oovs = max([len(ex.article_oovs) for ex in example_list])
    # Store the in-article OOVs themselves
    self.art_oovs = [ex.article_oovs for ex in example_list]
    # Store the version of the enc_batch that uses the article OOV ids
    self.enc_batch_extend_vocab = np.zeros((self.batch_size, max_enc_seq_len), dtype=np.int32)
    for i, ex in enumerate(example_list):
      self.enc_batch_extend_vocab[i, :] = ex.enc_input_extend_vocab[:]

  def init_decoder_seq(self, example_list):
    # Pad the inputs and targets
    for ex in example_list:
      ex.pad_decoder_inp_targ(max_dec_steps, self.pad_id)           #config

    # Initialize the numpy arrays.
    self.dec_batch = np.zeros((self.batch_size, max_dec_steps), dtype=np.int32)         #config
    self.target_batch = np.zeros((self.batch_size, max_dec_steps), dtype=np.int32)      #config
    # self.dec_padding_mask = np.zeros((self.batch_size, config.max_dec_steps), dtype=np.float32)
    self.dec_lens = np.zeros((self.batch_size), dtype=np.int32)

    # Fill in the numpy arrays
    for i, ex in enumerate(example_list):
      self.dec_batch[i, :] = ex.dec_input[:]
      self.target_batch[i, :] = ex.target[:]
      self.dec_lens[i] = ex.dec_len
      # for j in range(ex.dec_len):
      #   self.dec_padding_mask[i][j] = 1

  def store_orig_strings(self, example_list):
    self.original_articles = [ex.original_article for ex in example_list] # list of lists
    self.original_abstracts = [ex.original_abstract for ex in example_list] # list of lists
    self.original_abstracts_sents = [ex.original_abstract_sents for ex in example_list] # list of list of lists


#-------------------------------------------------------------------------------------------------
class Batcher(object):
  BATCH_QUEUE_MAX = max_batch_queue  #30 # max number of batches the batch_queue can hold

  def __init__(self, data_path, vocab, mode, batch_size, single_pass):
    self._data_path = data_path
    self._vocab = vocab
    self._single_pass = single_pass
    self.mode = mode
    self.batch_size = batch_size
    # Initialize a queue of Batches waiting to be used, and a queue of Examples waiting to be batched
    self._batch_queue = Queue.Queue(self.BATCH_QUEUE_MAX)
    self._example_queue = Queue.Queue(self.BATCH_QUEUE_MAX * self.batch_size)

    # Different settings depending on whether we're in single_pass mode or not
    if single_pass:
      self._num_example_q_threads = 1 # just one thread, so we read through the dataset just once
      self._num_batch_q_threads = 1  # just one thread to batch examples
      self._bucketing_cache_size = 1 # only load one batch's worth of examples before bucketing; this essentially means no bucketing
      self._finished_reading = False # this will tell us when we're finished reading the dataset
    else:
      self._num_example_q_threads = 1 #16 # num threads to fill example queue
      self._num_batch_q_threads = 1 #4  # num threads to fill batch queue
      self._bucketing_cache_size = 1 #100 # how many batches-worth of examples to load into cache before bucketing

    # Start the threads that load the queues
    self._example_q_threads = []
    for _ in range(self._num_example_q_threads):
      self._example_q_threads.append(Thread(target=self.fill_example_queue))
      self._example_q_threads[-1].daemon = True
      self._example_q_threads[-1].start()
    self._batch_q_threads = []
    for _ in range(self._num_batch_q_threads):
      self._batch_q_threads.append(Thread(target=self.fill_batch_queue))
      self._batch_q_threads[-1].daemon = True
      self._batch_q_threads[-1].start()

    # Start a thread that watches the other threads and restarts them if they're dead
    if not single_pass: # We don't want a watcher in single_pass mode because the threads shouldn't run forever
      self._watch_thread = Thread(target=self.watch_threads)
      self._watch_thread.daemon = True
      self._watch_thread.start()

  def next_batch(self):
    # If the batch queue is empty, print a warning
    if self._batch_queue.qsize() == 0:
      # tf.logging.warning('Bucket input queue is empty when calling next_batch. Bucket queue size: %i, Input queue size: %i', self._batch_queue.qsize(), self._example_queue.qsize())
      if self._single_pass and self._finished_reading:
        tf.compat.v1.logging.info("Finished reading dataset in single_pass mode.")
        return None

    batch = self._batch_queue.get() # get the next Batch
    return batch

  def fill_example_queue(self):
    input_gen = self.text_generator(example_generator(self._data_path, self._single_pass))

    while True:
      try:
        (article, abstract) = next(input_gen) # read the next example from file. article and abstract are both strings.
      except StopIteration: # if there are no more examples:
        tf.compat.v1.logging.info("The example generator for this example queue filling thread has exhausted data.")
        if self._single_pass:
          tf.compat.v1.logging.info("single_pass mode is on, so we've finished reading dataset. This thread is stopping.")
          self._finished_reading = True
          break
        else:
          raise Exception("single_pass mode is off but the example generator is out of data; error.")

      # abstract_sentences = [sent.strip() for sent in data.abstract2sents(abstract)] # Use the <s> and </s> tags in abstract to get a list of sentences.
      abstract_sentences = [abstract.strip()]
      example = Example(article, abstract_sentences, self._vocab) # Process into an Example.
      self._example_queue.put(example) # place the Example in the example queue.

  def fill_batch_queue(self):
    while True:
      if self.mode == 'decode':
        # beam search decode mode single example repeated in the batch
        ex = self._example_queue.get()
        b = [ex for _ in range(self.batch_size)]
        self._batch_queue.put(Batch(b, self._vocab, self.batch_size))
      else:
        # Get bucketing_cache_size-many batches of Examples into a list, then sort
        inputs = []
        for _ in range(self.batch_size * self._bucketing_cache_size):
          inputs.append(self._example_queue.get())
        inputs = sorted(inputs, key=lambda inp: inp.enc_len, reverse=True) # sort by length of encoder sequence

        # Group the sorted Examples into batches, optionally shuffle the batches, and place in the batch queue.
        batches = []
        for i in range(0, len(inputs), self.batch_size):
          batches.append(inputs[i:i + self.batch_size])
        if not self._single_pass:
          shuffle(batches)
        for b in batches:  # each b is a list of Example objects
          self._batch_queue.put(Batch(b, self._vocab, self.batch_size))

  def watch_threads(self):
    while True:
      tf.compat.v1.logging.info(
        'Bucket queue size: %i, Input queue size: %i',
        self._batch_queue.qsize(), self._example_queue.qsize())

      time.sleep(60)
      for idx,t in enumerate(self._example_q_threads):
        if not t.is_alive(): # if the thread is dead
          tf.compat.v1.logging.error('Found example queue thread dead. Restarting.')
          new_t = Thread(target=self.fill_example_queue)
          self._example_q_threads[idx] = new_t
          new_t.daemon = True
          new_t.start()
      for idx,t in enumerate(self._batch_q_threads):
        if not t.is_alive(): # if the thread is dead
          tf.compat.v1.logging.error('Found batch queue thread dead. Restarting.')
          new_t = Thread(target=self.fill_batch_queue)
          self._batch_q_threads[idx] = new_t
          new_t.daemon = True
          new_t.start()


  def text_generator(self, example_generator):
    while True:
      e = next(example_generator) # e is a tf.Example
      try:
        article_text = e.features.feature['article'].bytes_list.value[0] # the article text was saved under the key 'article' in the data files
        abstract_text = e.features.feature['abstract'].bytes_list.value[0] # the abstract text was saved under the key 'abstract' in the data files
        article_text = article_text.decode()
        abstract_text = abstract_text.decode()
      except ValueError:
        tf.compat.v1.logging.error('Failed to get article or abstract from example')
        continue
      if len(article_text)==0: # See https://github.com/abisee/pointer-generator/issues/1
        #tf.logging.warning('Found an example with empty article text. Skipping it.')
        continue
      else:
        yield (article_text, abstract_text)

##Model with global attention mechanism

In [0]:
'''
import torch as T
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
'''

def init_lstm_wt(lstm):                                 #initialisation 
    for name, _ in lstm.named_parameters():
        if 'weight' in name:
            wt = getattr(lstm, name)
            #print(wt)
            wt.data = wt.data.uniform_(-rand_unif_init_mag, rand_unif_init_mag)
            #wt.uniform_(-rand_unif_init_mag, rand_unif_init_mag)     #commit
        elif 'bias' in name:
            # set forget bias to 1
            bias = getattr(lstm, name)
            n = bias.size(0)
            start, end = n // 4, n // 2

            bias.data = bias.data.fill_(0.)
            #bias.fill_(0.)
            bias.data[start:end].fill_(1.)

def init_linear_wt(linear):                             #initialisation

    linear.weight.data = linear.weight.data.normal_(std=trunc_norm_init_std)
    #linear.weight.normal_(std=trunc_norm_init_std)
    if linear.bias is not None:
        linear.bias.data = linear.bias.data.normal_(std=trunc_norm_init_std)
        #linear.bias.normal_(std=trunc_norm_init_std)

def init_wt_normal(wt):                                 #initialisation
    wt.data = wt.data.normal_(std=trunc_norm_init_std)
    #wt.normal_(std=trunc_norm_init_std)


class Encoder(nn.Module):
    """
    bi-directional LSTM, reduced linear layer
    """
    def __init__(self):
        super(Encoder, self).__init__()

        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)    #batch_true: If True, then the input and output tensors are provided as (batch, seq, feature)
        init_lstm_wt(self.lstm)

        self.reduce_h = nn.Linear(hidden_dim * 2, hidden_dim)        #hidden state 
        init_linear_wt(self.reduce_h)
        self.reduce_c = nn.Linear(hidden_dim * 2, hidden_dim)
        init_linear_wt(self.reduce_c)

    def forward(self, x, seq_lens):
        """
        param  x:  input sequence
        param  seq_len:  length of sequence

        return:
        enc_out:  Outputs of the encoder for all time steps (batch_size, length_input_sequence, 2*hidden_size)
        (h_reduced, c_reduced):  Tuple containing final hidden state & cell state of encoder. Shape of h & c: (batch_size, hidden_size)
        """
        packed = pack_padded_sequence(x, seq_lens, batch_first=True)    #accepts any input that has at least two dimensions. You can apply it to pack the labels, and use the output of the RNN with them to compute the loss directly
        enc_out, enc_hid = self.lstm(packed)                            # tensor containing the hidden state for t = seq_len. and tensor containing the cell state for t = seq_len.
        enc_out,_ = pad_packed_sequence(enc_out, batch_first=True)      
        enc_out = enc_out.contiguous()                              #bs, n_seq, 2*n_hid
        h, c = enc_hid                                              #shape of h: 2, bs, n_hid
        h = T.cat(list(h), dim=1)                                   #bs, 2*n_hid
        c = T.cat(list(c), dim=1)
        h_reduced = F.relu(self.reduce_h(h))                        #bs,n_hid
        c_reduced = F.relu(self.reduce_c(c))
        return enc_out, (h_reduced, c_reduced)      #enc_out: all encoder


class encoder_attention(nn.Module):

    def __init__(self):
        super(encoder_attention, self).__init__()
        self.W_h = nn.Linear(hidden_dim * 2, hidden_dim * 2, bias=False)    #no bias just the weight matrix
        self.W_s = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.v = nn.Linear(hidden_dim * 2, 1, bias=False)


    def forward(self, st_hat, h, enc_padding_mask, sum_temporal_srcs):              #INTRA-TEMPORAL ATTENTION ON INPUT SEQUENCE in paper?
        ''' Perform attention over encoder hidden states
        :param st_hat: decoder hidden state at current time step
        :param h: encoder hidden states
        :param enc_padding_mask:
        :param sum_temporal_srcs: if using intra-temporal attention, contains summation of attention weights from previous decoder time steps
        return
        ct_e: encoder context vector
        at:   attention weight
        sum_temporal_srcs:  sum of attention weights from previous decoder time steps, will need to be input again for next iteration
        '''

        # Standard attention technique (eq 1 in https://arxiv.org/pdf/1704.04368.pdf)
        et = self.W_h(h)                        # bs,n_seq,2*n_hid
        dec_fea = self.W_s(st_hat).unsqueeze(1) # bs,1,2*n_hid
        et = et + dec_fea
        et = T.tanh(et)                         # bs,n_seq,2*n_hid
        et = self.v(et).squeeze(2)              # bs,n_seq

        # intra-temporal attention     (eq 3 in https://arxiv.org/pdf/1705.04304.pdf)
        if intra_encoder:
            exp_et = T.exp(et)
            if sum_temporal_srcs is None:
                et1 = exp_et
                sum_temporal_srcs  = get_cuda(T.FloatTensor(et.size()).fill_(1e-10)) + exp_et           #defined temporal score
            else:
                et1 = exp_et/sum_temporal_srcs  #bs, n_seq
                sum_temporal_srcs = sum_temporal_srcs + exp_et
        else:
            et1 = F.softmax(et, dim=1)

        # assign 0 probability for padded elements
        at = et1 * enc_padding_mask
        normalization_factor = at.sum(1, keepdim=True)
        at = at / normalization_factor

        at = at.unsqueeze(1)                    #bs,1,n_seq
        # Compute encoder context vector
        ct_e = T.bmm(at, h)                     #bs, 1, 2*n_hid, batch matrix-matrix product of matrices (temporal score and encoder hidden state)
        ct_e = ct_e.squeeze(1)
        at = at.squeeze(1)

        return ct_e, at, sum_temporal_srcs

class decoder_attention(nn.Module):
    def __init__(self):
        super(decoder_attention, self).__init__()
        if intra_decoder:
            self.W_prev = nn.Linear(hidden_dim, hidden_dim, bias=False)        #weight
            self.W_s = nn.Linear(hidden_dim, hidden_dim)
            self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, s_t, prev_s):
        '''Perform intra_decoder attention
        Args
        :param s_t: hidden state of decoder at current time step
        :param prev_s: If intra_decoder attention, contains list of previous decoder hidden states
        '''
        if intra_decoder is False:
            ct_d = get_cuda(T.zeros(s_t.size()))
        elif prev_s is None:
            ct_d = get_cuda(T.zeros(s_t.size()))
            prev_s = s_t.unsqueeze(1)               #bs, 1, n_hid
        else:
            # Standard attention technique (eq 1 in https://arxiv.org/pdf/1704.04368.pdf)      e = v tanh(Wh + Ws + b)
            et = self.W_prev(prev_s)                # bs,t-1,n_hid
            dec_fea = self.W_s(s_t).unsqueeze(1)    # bs,1,n_hid
            et = et + dec_fea
            et = T.tanh(et)                         # bs,t-1,n_hid
            et = self.v(et).squeeze(2)              # bs,t-1
            # intra-decoder attention     (eq 7 & 8 in https://arxiv.org/pdf/1705.04304.pdf)
            at = F.softmax(et, dim=1).unsqueeze(1)  #bs, 1, t-1,  alpha
            ct_d = T.bmm(at, prev_s).squeeze(1)     #bs, n_hid
            prev_s = T.cat([prev_s, s_t.unsqueeze(1)], dim=1)    #bs, t, n_hid, keep adding previous hidden state 

        return ct_d, prev_s                 #decoder context vector, previous decoder hidden states


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.enc_attention = encoder_attention()
        self.dec_attention = decoder_attention()
        self.x_context = nn.Linear(hidden_dim*2 + emb_dim, emb_dim)

        self.lstm = nn.LSTMCell(emb_dim, hidden_dim)
        init_lstm_wt(self.lstm)

        self.p_gen_linear = nn.Linear(hidden_dim * 5 + emb_dim, 1)

        #p_vocab
        self.V = nn.Linear(hidden_dim*4, hidden_dim)
        self.V1 = nn.Linear(hidden_dim, vocab_size)
        init_linear_wt(self.V1)

    def forward(self, x_t, s_t, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s):
        x = self.x_context(T.cat([x_t, ct_e], dim=1))
        s_t = self.lstm(x, s_t)

        dec_h, dec_c = s_t
        st_hat = T.cat([dec_h, dec_c], dim=1)
        ct_e, attn_dist, sum_temporal_srcs = self.enc_attention(st_hat, enc_out, enc_padding_mask, sum_temporal_srcs)

        ct_d, prev_s = self.dec_attention(dec_h, prev_s)        #intra-decoder attention

        p_gen = T.cat([ct_e, ct_d, st_hat, x], 1)
        p_gen = self.p_gen_linear(p_gen)            # bs,1
        p_gen = T.sigmoid(p_gen)                    # bs,1

        out = T.cat([dec_h, ct_e, ct_d], dim=1)     # bs, 4*n_hid
        out = self.V(out)                           # bs,n_hid
        out = self.V1(out)                          # bs, n_vocab
        vocab_dist = F.softmax(out, dim=1)
        vocab_dist = p_gen * vocab_dist
        attn_dist_ = (1 - p_gen) * attn_dist

        # pointer mechanism (as suggested in eq 9 https://arxiv.org/pdf/1704.04368.pdf)
        if extra_zeros is not None:
            vocab_dist = T.cat([vocab_dist, extra_zeros], dim=1)
        final_dist = vocab_dist.scatter_add(1, enc_batch_extend_vocab, attn_dist_)

        return final_dist, s_t, ct_e, sum_temporal_srcs, prev_s



class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.embeds = nn.Embedding(vocab_size, emb_dim)
        init_wt_normal(self.embeds.weight)

        self.encoder = get_cuda(self.encoder)
        self.decoder = get_cuda(self.decoder)
        self.embeds = get_cuda(self.embeds)





## Utility

In [0]:
import numpy as np
import torch as T


def get_cuda(tensor):
    if T.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

def get_enc_data(batch):
    batch_size = len(batch.enc_lens)
    enc_batch = T.from_numpy(batch.enc_batch).long()
    enc_padding_mask = T.from_numpy(batch.enc_padding_mask).float()

    enc_lens = batch.enc_lens

    ct_e = T.zeros(batch_size, 2*   hidden_dim)   #config.hidden_dim

    enc_batch = get_cuda(enc_batch)
    enc_padding_mask = get_cuda(enc_padding_mask)

    ct_e = get_cuda(ct_e)

    enc_batch_extend_vocab = None
    if batch.enc_batch_extend_vocab is not None:
        enc_batch_extend_vocab = T.from_numpy(batch.enc_batch_extend_vocab).long()
        enc_batch_extend_vocab = get_cuda(enc_batch_extend_vocab)

    extra_zeros = None
    if batch.max_art_oovs > 0:
        extra_zeros = T.zeros(batch_size, batch.max_art_oovs)
        extra_zeros = get_cuda(extra_zeros)


    return enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, ct_e


def get_dec_data(batch):
    dec_batch = T.from_numpy(batch.dec_batch).long()
    dec_lens = batch.dec_lens
    max_dec_len = np.max(dec_lens)
    dec_lens = T.from_numpy(batch.dec_lens).float()

    target_batch = T.from_numpy(batch.target_batch).long()

    dec_batch = get_cuda(dec_batch)
    dec_lens = get_cuda(dec_lens)
    target_batch = get_cuda(target_batch)

    return dec_batch, max_dec_len, dec_lens, target_batch

## Beam search

In [0]:
import numpy as np
import torch as T


class Beam(object):
    def __init__(self, start_id, end_id, unk_id, hidden_state, context):
        h,c = hidden_state                                              #(n_hid,)
        self.tokens = T.LongTensor(beam_size,1).fill_(start_id)  #(beam, t) after t time steps
        self.scores = T.FloatTensor(beam_size,1).fill_(-30)      #beam,1; Initial score of beams = -30
        self.tokens, self.scores = get_cuda(self.tokens), get_cuda(self.scores)
        self.scores[0][0] = 0                                           #At time step t=0, all beams should extend from a single beam. So, I am giving high initial score to 1st beam
        self.hid_h = h.unsqueeze(0).repeat(beam_size, 1)         #beam, n_hid
        self.hid_c = c.unsqueeze(0).repeat(beam_size, 1)         #beam, n_hid
        self.context = context.unsqueeze(0).repeat(beam_size, 1) #beam, 2*n_hid
        self.sum_temporal_srcs = None
        self.prev_s = None
        self.done = False
        self.end_id = end_id
        self.unk_id = unk_id

    def get_current_state(self):
        tokens = self.tokens[:,-1].clone()
        for i in range(len(tokens)):
            if tokens[i].item() >= vocab_size:
                tokens[i] = self.unk_id
        return tokens


    def advance(self, prob_dist, hidden_state, context, sum_temporal_srcs, prev_s):
        '''Perform beam search: Considering the probabilites of given n_beam x n_extended_vocab words, select first n_beam words that give high total scores
        :param prob_dist: (beam, n_extended_vocab)
        :param hidden_state: Tuple of (beam, n_hid) tensors
        :param context:   (beam, 2*n_hidden)
        :param sum_temporal_srcs:   (beam, n_seq)
        :param prev_s:  (beam, t, n_hid)
        '''
        n_extended_vocab = prob_dist.size(1)
        h, c = hidden_state
        log_probs = T.log(prob_dist+eps)                         #beam, n_extended_vocab

        scores = log_probs + self.scores                                #beam, n_extended_vocab
        scores = scores.view(-1,1)                                      #beam*n_extended_vocab, 1
        best_scores, best_scores_id = T.topk(input=scores, k=beam_size, dim=0)   #will be sorted in descending order of scores
        self.scores = best_scores                                       #(beam,1); sorted
        beams_order = best_scores_id.squeeze(1)/n_extended_vocab        #(beam,); sorted
        best_words = best_scores_id%n_extended_vocab                    #(beam,1); sorted
        self.hid_h = h[beams_order]                                     #(beam, n_hid); sorted
        self.hid_c = c[beams_order]                                     #(beam, n_hid); sorted
        self.context = context[beams_order]
        if sum_temporal_srcs is not None:
            self.sum_temporal_srcs = sum_temporal_srcs[beams_order]     #(beam, n_seq); sorted
        if prev_s is not None:
            self.prev_s = prev_s[beams_order]                           #(beam, t, n_hid); sorted
        self.tokens = self.tokens[beams_order]                          #(beam, t); sorted
        self.tokens = T.cat([self.tokens, best_words], dim=1)           #(beam, t+1); sorted

        #End condition is when top-of-beam is EOS.
        if best_words[0][0] == self.end_id:
            self.done = True

    def get_best(self):
        best_token = self.tokens[0].cpu().numpy().tolist()              #Since beams are always in sorted (descending) order, 1st beam is the best beam
        try:
            end_idx = best_token.index(self.end_id)
        except ValueError:
            end_idx = len(best_token)
        best_token = best_token[1:end_idx]
        return best_token

    def get_all(self):
        all_tokens = []
        for i in range(len(self.tokens)):
            all_tokens.append(self.tokens[i].cpu().numpy())
        return all_tokens


def beam_search(enc_hid, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, model, start_id, end_id, unk_id):

    batch_size = len(enc_hid[0])
    beam_idx = T.LongTensor(list(range(batch_size)))
    beams = [Beam(start_id, end_id, unk_id, (enc_hid[0][i], enc_hid[1][i]), ct_e[i]) for i in range(batch_size)]   #For each example in batch, create Beam object
    n_rem = batch_size                                                  #Index of beams that are active, i.e: didn't generate [STOP] yet
    sum_temporal_srcs = None                                            #Number of examples in batch that didn't generate [STOP] yet
    prev_s = None

    for t in range(max_dec_steps):
        x_t = T.stack(
            [beam.get_current_state() for beam in beams if beam.done == False]      #remaining(rem),beam
        ).contiguous().view(-1)                                                     #(rem*beam,)
        x_t = model.embeds(x_t)                                                 #rem*beam, n_emb

        dec_h = T.stack(
            [beam.hid_h for beam in beams if beam.done == False]                    #rem*beam,n_hid
        ).contiguous().view(-1,hidden_dim)
        dec_c = T.stack(
            [beam.hid_c for beam in beams if beam.done == False]                    #rem,beam,n_hid
        ).contiguous().view(-1,hidden_dim)                                   #rem*beam,n_hid

        ct_e = T.stack(
            [beam.context for beam in beams if beam.done == False]                  #rem,beam,n_hid
        ).contiguous().view(-1,2*hidden_dim)                                 #rem,beam,n_hid

        if sum_temporal_srcs is not None:
            sum_temporal_srcs = T.stack(
                [beam.sum_temporal_srcs for beam in beams if beam.done == False]
            ).contiguous().view(-1, enc_out.size(1))                                #rem*beam, n_seq

        if prev_s is not None:
            prev_s = T.stack(
                [beam.prev_s for beam in beams if beam.done == False]
            ).contiguous().view(-1, t, hidden_dim)                           #rem*beam, t-1, n_hid


        s_t = (dec_h, dec_c)
        enc_out_beam = enc_out[beam_idx].view(n_rem,-1).repeat(1, beam_size).view(-1, enc_out.size(1), enc_out.size(2))
        enc_pad_mask_beam = enc_padding_mask[beam_idx].repeat(1, beam_size).view(-1, enc_padding_mask.size(1))

        extra_zeros_beam = None
        if extra_zeros is not None:
            extra_zeros_beam = extra_zeros[beam_idx].repeat(1, beam_size).view(-1, extra_zeros.size(1))
        enc_extend_vocab_beam = enc_batch_extend_vocab[beam_idx].repeat(1, beam_size).view(-1, enc_batch_extend_vocab.size(1))

        final_dist, (dec_h, dec_c), ct_e, sum_temporal_srcs, prev_s = model.decoder(x_t, s_t, enc_out_beam, enc_pad_mask_beam, ct_e, extra_zeros_beam, enc_extend_vocab_beam, sum_temporal_srcs, prev_s)              #final_dist: rem*beam, n_extended_vocab

        final_dist = final_dist.view(n_rem, beam_size, -1)                   #final_dist: rem, beam, n_extended_vocab
        dec_h = dec_h.view(n_rem, beam_size, -1)                             #rem, beam, n_hid
        dec_c = dec_c.view(n_rem, beam_size, -1)                             #rem, beam, n_hid
        ct_e = ct_e.view(n_rem, beam_size, -1)                             #rem, beam, 2*n_hid

        if sum_temporal_srcs is not None:
            sum_temporal_srcs = sum_temporal_srcs.view(n_rem, beam_size, -1) #rem, beam, n_seq

        if prev_s is not None:
            prev_s = prev_s.view(n_rem, beam_size, -1, hidden_dim)    #rem, beam, t

        # For all the active beams, perform beam search
        active = []         #indices of active beams after beam search

        for i in range(n_rem):
            b = beam_idx[i].item()
            beam = beams[b]
            if beam.done:
                continue

            sum_temporal_srcs_i = prev_s_i = None
            if sum_temporal_srcs is not None:
                sum_temporal_srcs_i = sum_temporal_srcs[i]                              #beam, n_seq
            if prev_s is not None:
                prev_s_i = prev_s[i]                                                #beam, t, n_hid
            beam.advance(final_dist[i], (dec_h[i], dec_c[i]), ct_e[i], sum_temporal_srcs_i, prev_s_i)
            if beam.done == False:
                active.append(b)

        if len(active) == 0:
            break

        beam_idx = T.LongTensor(active)
        n_rem = len(beam_idx)

    predicted_words = []
    for beam in beams:
        predicted_words.append(beam.get_best())

    return predicted_words

## Evaluation with ROUGE

In [0]:
'''
import os

import time

import torch as T
import torch.nn as nn
import torch.nn.functional as F


from rouge import Rouge
import argparse
'''

def get_cuda(tensor):
    if T.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

class Evaluate_r(object):
    def __init__(self, data_path, opt, batch_size):
        self.vocab = Vocab(vocab_path, vocab_size)
        self.batcher = Batcher(data_path, self.vocab, mode='eval',
                               batch_size=batch_size, single_pass=True)
        self.opt = opt
        time.sleep(5)

    def setup_valid(self):
        self.model = Model()
        self.model = get_cuda(self.model)
        checkpoint = T.load(os.path.join(save_model_path, self.opt.load_model))
        self.model.load_state_dict(checkpoint["model_dict"])


    def print_original_predicted(self, decoded_sents, ref_sents, article_sents, loadfile):
        filename = "test_"+loadfile.split(".")[0]+".txt"
    
        with open(os.path.join(save_example_path,filename), "w") as f:
            for i in range(len(decoded_sents)):
                f.write("article: "+article_sents[i] + "\n")
                f.write("ref: " + ref_sents[i] + "\n")
                f.write("dec: " + decoded_sents[i] + "\n\n")

    def evaluate_batch(self, print_sents = False):

        self.setup_valid()
        batch = self.batcher.next_batch()
        start_id = self.vocab.word2id(START_DECODING)
        end_id = self.vocab.word2id(STOP_DECODING)
        unk_id = self.vocab.word2id(UNKNOWN_TOKEN)
        decoded_sents = []
        ref_sents = []
        article_sents = []
        rouge = Rouge()
        while batch is not None:
            enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, ct_e = get_enc_data(batch)

            with T.autograd.no_grad():
                enc_batch = self.model.embeds(enc_batch)
                enc_out, enc_hidden = self.model.encoder(enc_batch, enc_lens)

            #-----------------------Summarization----------------------------------------------------
            with T.autograd.no_grad():
                pred_ids = beam_search(enc_hidden, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, self.model, start_id, end_id, unk_id)

            for i in range(len(pred_ids)):
                decoded_words = outputids2words(pred_ids[i], self.vocab, batch.art_oovs[i])
                if len(decoded_words) < 2:
                    decoded_words = "xxx"
                else:
                    decoded_words = " ".join(decoded_words)
                decoded_sents.append(decoded_words)
                abstract = batch.original_abstracts[i]
                article = batch.original_articles[i]
                ref_sents.append(abstract)
                article_sents.append(article)

            batch = self.batcher.next_batch()

        load_file = self.opt.load_model

        if print_sents:
            self.print_original_predicted(decoded_sents, ref_sents, article_sents, load_file)

        scores = rouge.get_scores(decoded_sents, ref_sents, avg = True)
        if self.opt.task == "test":
            print(load_file, "scores:", scores)
        else:
            rouge_l = scores["rouge-l"]["f"]
            print(load_file, "rouge_l:", "%.4f" % rouge_l)
            print('Scores:', scores)
        
        rouge_1 = scores['rouge-1']['f']        #rouge 1
        rouge_2 = scores['rouge-2']['f']        #rouge 2
        rouge_l = scores['rouge-l']['f']        #rouge L

        return rouge_1, rouge_2, rouge_l




## Evaluation with BERTScore

In [0]:
import os

import time

import torch as T
import torch.nn as nn
import torch.nn.functional as F

import gc
import bert_score
import argparse

def get_cuda(tensor):
    if T.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

class Evaluate_b(object):
    def __init__(self, data_path, opt, batch_size):
        self.vocab = Vocab(vocab_path, vocab_size)
        self.batcher = Batcher(data_path, self.vocab, mode='eval',
                               batch_size=batch_size, single_pass=True)
        self.opt = opt

        time.sleep(5)

    def setup_valid(self):
        self.model = Model()
        self.model = get_cuda(self.model)
        checkpoint = T.load(os.path.join(save_model_path, self.opt.load_model))
        self.model.load_state_dict(checkpoint["model_dict"])


    def print_original_predicted(self, decoded_sents, ref_sents, article_sents, loadfile):
        filename = "test_"+loadfile.split(".")[0]+".txt"
    
        with open(os.path.join(save_example_path,filename), "w") as f:
            for i in range(len(decoded_sents)):
                f.write("article: "+article_sents[i] + "\n")
                f.write("ref: " + ref_sents[i] + "\n")
                f.write("dec: " + decoded_sents[i] + "\n\n")

    def evaluate_batch(self, print_sents = False):

        self.setup_valid()
        batch = self.batcher.next_batch()
        start_id = self.vocab.word2id(START_DECODING)
        end_id = self.vocab.word2id(STOP_DECODING)
        unk_id = self.vocab.word2id(UNKNOWN_TOKEN)
        decoded_sents = []
        ref_sents = []
        article_sents = []
        #rouge = Rouge()
        while batch is not None:
            enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, ct_e = get_enc_data(batch)

            with T.autograd.no_grad():
                enc_batch = self.model.embeds(enc_batch)
                enc_out, enc_hidden = self.model.encoder(enc_batch, enc_lens)

            #-----------------------Summarization----------------------------------------------------
            with T.autograd.no_grad():
                pred_ids = beam_search(enc_hidden, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, self.model, start_id, end_id, unk_id)

            for i in range(len(pred_ids)):
                decoded_words = outputids2words(pred_ids[i], self.vocab, batch.art_oovs[i])
                if len(decoded_words) < 2:
                    decoded_words = "xxx"
                else:
                    decoded_words = " ".join(decoded_words)
                decoded_sents.append(decoded_words)
                abstract = batch.original_abstracts[i]
                article = batch.original_articles[i]
                ref_sents.append(abstract)
                article_sents.append(article)

            batch = self.batcher.next_batch()

        load_file = self.opt.load_model

        _,_,f = bert_score.score(decoded_sents, ref_sents, lang='en', verbose = False)

        print('{}  score:{}'.format(load_file, f.mean()))
        gc.collect()
        
        if print_sents:
            self.print_original_predicted(decoded_sents, ref_sents, article_sents, load_file)

        return f.mean()
        


#Experiement

##Gigaword testing

testing models folder:  

In [0]:
model_folder = 'drive/My Drive/Gigaword testing'        #directory containing all testing models for Gigaword dataset

In [62]:
!ls 'drive/My Drive/Gigaword testing' 

'hybrid testing'  'RL(b) testing model'   test_00.bin
'ml testing'	  'RL(r) testing model'   vocab


###Hyperparameter

In [0]:
test_data_path = (model_folder + '/test_00.bin')	   #'drive/My Drive/Gigaword/giga/chunked/test/test_00.bin'         #testing data file path
vocab_path =   (model_folder + '/vocab')		#'drive/My Drive/Gigaword/giga/vocab'                           #vocab data file path

# Hyperparameters
hidden_dim = 512
emb_dim = 256
batch_size = 200
max_enc_steps = 55
max_dec_steps = 15
beam_size = 4
min_dec_steps= 3
vocab_size = 50000

lr = 0.001
rand_unif_init_mag = 0.02
trunc_norm_init_std = 1e-4

eps = 1e-12
max_iterations = 16000
max_batch_queue = 1000          #1000


intra_encoder = True
intra_decoder = True

In [0]:
!ls 'drive/My Drive/Gigaword'

 giga				'ml testing'	        sumdata
 giga_pretrain.tar		'RL(b) testing model'   summary.tar.gz
'local attention example d100'	'RL(r) testing model'


###ML testing(ROUGE+BERTScore)

In [0]:
save_model_path = (model_folder + '/ml testing')#'drive/My Drive/Gigaword/ml testing'              #directory path for the model need to be tested
model_name_for_r_testing = 'ML testing.tar'
model_name_for_b_testing = 'ML testing.tar'                     #both names are the same for ML testing

In [71]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()


    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_ml_1, r_ml_2, r_ml_L = eval_r.evaluate_batch()

    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    ml_bert_testing = eval_b.evaluate_batch()







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
ML testing.tar scores: {'rouge-1': {'f': 0.4211750260014512, 'p': 0.45982025514168373, 'r': 0.40514399771993204}, 'rouge-2': {'f': 0.22046333434940685, 'p': 0.24310209433423713, 'r': 0.21108568415711254}, 'rouge-l': {'f': 0.40631281347425713, 'p': 0.44417428999571834, 'r': 0.38991643404079745}}






max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Finished reading dataset in single_pass mode.
ML testing.tar  score:0.9029324650764465


In [72]:
print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_ml_1, r_ml_2, r_ml_L, ml_bert_testing))

ROUGE_1: 0.4212 ; ROUGE_2: 0.2205; ROUGE_L: 0.4063; BERTScore: 0.9029


###Hybrid testing (ROUGE+BERTScore)

In [0]:
save_model_path = (model_folder + '/hybrid testing')        #'drive/My Drive/Gigaword/hybrid testing'              #directory path for the model need to be tested
model_name_for_r_testing = 'hybrid for ROUGE testing.tar'
model_name_for_b_testing = 'hybrid for BERT testing.tar'                    

In [74]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()

    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_hy_1, r_hy_2, r_hy_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    hy_bert_testing = eval_b.evaluate_batch()







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
hybrid for ROUGE testing.tar scores: {'rouge-1': {'f': 0.4232801800481467, 'p': 0.462242559028273, 'r': 0.4070486371398023}, 'rouge-2': {'f': 0.2223031150585183, 'p': 0.24536857190428593, 'r': 0.21281178841000253}, 'rouge-l': {'f': 0.4080882263595928, 'p': 0.44618194503908803, 'r': 0.3914904075696028}}






max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Finished reading dataset in single_pass mode.
hybrid for BERT testing.tar  score:0.9031652808189392


In [75]:
print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_hy_1, r_hy_2, r_hy_L, hy_bert_testing))

ROUGE_1: 0.4233 ; ROUGE_2: 0.2223; ROUGE_L: 0.4081; BERTScore: 0.9032


###RL(r) testing(ROUGE+BERTScore)

In [0]:
save_model_path =  (model_folder + '/RL(r) testing model')             #'drive/My Drive/Gigaword/RL(r) testing model'              #directory path for the RL(r) model need to be tested
model_name_for_r_testing = 'RL(r) for ROUGE testing.tar'
model_name_for_b_testing = 'RL(r) for BERT testing.tar'                    

In [98]:
!ls 'drive/My Drive/Gigaword/RL(r) testing model'

'giga RL(r) BERT testing.tar'  'RL(r) Giga model for ROUGE testing.tar'
'RL(r) for BERT testing.tar'


In [77]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()


    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_rlr_1, r_rlr_2, r_rlr_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    rlr_bert_testing = eval_b.evaluate_batch()








max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(r) for ROUGE testing.tar scores: {'rouge-1': {'f': 0.32906548014053744, 'p': 0.2719791472655776, 'r': 0.4463953833097463}, 'rouge-2': {'f': 0.15910624180178037, 'p': 0.13320390184045677, 'r': 0.2177210814978669}, 'rouge-l': {'f': 0.43262097102848707, 'p': 0.45848843418486085, 'r': 0.4304524925630909}}






max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(r) for BERT testing.tar  score:0.8767116665840149


In [78]:
print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_rlr_1, r_rlr_2, r_rlr_L, rlr_bert_testing))

ROUGE_1: 0.3291 ; ROUGE_2: 0.1591; ROUGE_L: 0.4326; BERTScore: 0.8767


###RL(b) testing(ROUGE+BERTScore)

In [0]:
save_model_path =  (model_folder + '/RL(b) testing model')         #'drive/My Drive/Gigaword/RL(b) testing model'              #directory path for the model need to be tested
model_name_for_r_testing = 'RL(b) for ROUGE testing.tar'
model_name_for_b_testing = 'RL(b) for BERT testing.tar'                  

In [80]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()

    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_rlb_1, r_rlb_2, r_rlb_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    rlb_bert_testing = eval_b.evaluate_batch()







max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(b) for ROUGE testing.tar scores: {'rouge-1': {'f': 0.42036605420965195, 'p': 0.44055116906902614, 'r': 0.4218861143218181}, 'rouge-2': {'f': 0.2091989906511146, 'p': 0.22080562889491437, 'r': 0.2096222497740352}, 'rouge-l': {'f': 0.4033836281461028, 'p': 0.4248145188145189, 'r': 0.40202342951964265}}






max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: malignaggi
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(b) for BERT testing.tar  score:0.9044777154922485


In [81]:
print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_rlb_1, r_rlb_2, r_rlb_L, rlb_bert_testing))

ROUGE_1: 0.4204 ; ROUGE_2: 0.2092; ROUGE_L: 0.4034; BERTScore: 0.9045


###Table

In [82]:
data_frame = {'ROUGE 1':[r_ml_1, r_hy_1, r_rlr_1, r_rlb_1], 'ROUGE 2':[r_ml_2, r_hy_2, r_rlr_2, r_rlb_2], 'ROUGE L':[r_ml_L, r_hy_L, r_rlr_L, r_rlb_L], 
              'BERTScore':[float(ml_bert_testing), float(hy_bert_testing), float(rlr_bert_testing), float(rlb_bert_testing)]}
df = pd.DataFrame(data_frame, index= ['ML', 'Hybrid', 'RL(r)', 'RL(b)']).round(4)
df

,ROUGE 1,ROUGE 2,ROUGE L,BERTScore
ML,0.4212,0.2205,0.4063,0.9029
Hybrid,0.4233,0.2223,0.4081,0.9032
RL(r),0.3291,0.1591,0.4326,0.8767
RL(b),0.4204,0.2092,0.4034,0.9045


##CNN/DM testing

In [0]:
model_folder = 'drive/My Drive/CNNDM testing'        #directory containing all testing models for Gigaword dataset

In [17]:
!ls 'drive/My Drive/CNNDM testing' 

'hybrid testing'  'RL(b) testing'  'testing files'
'ml testing'	  'RL(r) testing'   vocab


###Hyperparameter

In [0]:
test_data_path = (model_folder + '/testing files/test_0000.bin')	 #"drive/My Drive/NLPProject/chunked/main_test/test_*"            #this is where you save the testing data, _* refers to multiple testing files
vocab_path = 	(model_folder+ '/vocab')	    #"drive/My Drive/NLPProject/vocab"                               #this is where you save the vocab file


# Hyperparameters
hidden_dim = 400
emb_dim = 200
batch_size = 50
max_enc_steps = 400		
max_dec_steps = 100		
beam_size = 5
min_dec_steps= 3
vocab_size = 30000 

lr = 0.001
rand_unif_init_mag = 0.02
trunc_norm_init_std = 1e-4

eps = 1e-12
max_iterations = 10000
max_batch_queue = 100

intra_encoder = True
intra_decoder = True

###ML testing(ROUGE+BERTScore)

In [0]:
save_model_path = (model_folder + '/ml testing')#'drive/My Drive/Gigaword/ml testing'              #directory path for the model need to be tested
model_name_for_r_testing = 'ml for ROUGE testing.tar'
model_name_for_b_testing = 'ml for BERT testing.tar'                     

In [159]:
!ls 'drive/My Drive/CNNDM testing/ml testing'  

'ml for BERT testing.tar'  'ml for ROUGE testing.tar'


In [20]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()


    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_ml_1, r_ml_2, r_ml_L = eval_r.evaluate_batch()


    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    ml_bert_testing = eval_b.evaluate_batch()

    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_ml_1, r_ml_2, r_ml_L, ml_bert_testing))



max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: moles
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


RuntimeError: ignored

In [21]:
T.cuda.is_available()

False

###Hybrid testing (ROUGE+BERTScore)

In [0]:
save_model_path = (model_folder + '/hybrid testing')        #'drive/My Drive/Gigaword/hybrid testing'              #directory path for the model need to be tested
model_name_for_r_testing = 'hybrid for ROUGE testing.tar'
model_name_for_b_testing = 'hybrid for BERT testing.tar'                

In [114]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()

    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_hy_1, r_hy_2, r_hy_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    hy_bert_testing = eval_b.evaluate_batch()
    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_hy_1, r_hy_2, r_hy_L, hy_bert_testing))



max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: moles
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
hybrid for ROUGE testing.tar scores: {'rouge-1': {'f': 0.3205578683941674, 'p': 0.2902061358414377, 'r': 0.386230365415494}, 'rouge-2': {'f': 0.12501077585092013, 'p': 0.11408270655704755, 'r': 0.14944289404443603}, 'rouge-l': {'f': 0.3322952610845442, 'p': 0.3149026005276578, 'r': 0.37588170544612104}}


max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: moles
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Finished reading dataset in single_pass mode.
hybrid for BERT testing.tar  score:0.8376993536949158
ROUGE_1: 0.3206 ; ROUGE_2: 0.1250; ROUGE_L: 0.3323; BERTScore: 0.8377


###RL(r) testing(ROUGE+BERTScore)

In [0]:
save_model_path =  (model_folder + '/RL(r) testing')             #'drive/My Drive/Gigaword/RL(r) testing model'              #directory path for the RL(r) model need to be tested
model_name_for_r_testing = 'RL(r) testing.tar'
model_name_for_b_testing = 'RL(r) testing.tar'                     #both names are the same for RL(r) testing

In [116]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()


    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_rlr_1, r_rlr_2, r_rlr_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    rlr_bert_testing = eval_b.evaluate_batch()

    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_rlr_1, r_rlr_2, r_rlr_L, rlr_bert_testing))



max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: moles
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(r) testing.tar scores: {'rouge-1': {'f': 0.2803087243540569, 'p': 0.23769121128051424, 'r': 0.36855667626067484}, 'rouge-2': {'f': 0.10360442582774361, 'p': 0.0877066415265804, 'r': 0.13673986648661476}, 'rouge-l': {'f': 0.3442404833958199, 'p': 0.35383596696042047, 'r': 0.3558461987570033}}


max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: moles
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(r) testing.tar  score:0.8204892873764038
ROUGE_1: 0.2803 ; ROUGE_2: 0.1036; ROUGE_L: 0.3442; BERTScore: 0.8205


###RL(b) testing(ROUGE+BERTScore)

In [0]:
save_model_path =  (model_folder + '/RL(b) testing')         #'drive/My Drive/Gigaword/RL(b) testing model'              #directory path for the model need to be tested
model_name_for_r_testing = 'RL(b) for ROUGE testing.tar'
model_name_for_b_testing = 'RL(b) for BERT testing.tar'                  

In [118]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()

    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_rlb_1, r_rlb_2, r_rlb_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    rlb_bert_testing = eval_b.evaluate_batch()
    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_rlb_1, r_rlb_2, r_rlb_L, rlb_bert_testing))



max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: moles
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(b) for ROUGE testing.tar scores: {'rouge-1': {'f': 0.30220935285726275, 'p': 0.32614719246397106, 'r': 0.30105528923872993}, 'rouge-2': {'f': 0.11736897732306088, 'p': 0.12741766778355326, 'r': 0.11691574551657218}, 'rouge-l': {'f': 0.32130715919253644, 'p': 0.35224349234967567, 'r': 0.31158168062560326}}


max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: moles
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Finished reading dataset in single_pass mode.
RL(b) for BERT testing.tar  score:0.8394527435302734
ROUGE_1: 0.3022 ; ROUGE_2: 0.1174; ROUGE_L: 0.3213; BERTScore: 0.8395


###Table

In [119]:
data_frame = {'ROUGE 1':[r_ml_1, r_hy_1, r_rlr_1, r_rlb_1], 'ROUGE 2':[r_ml_2, r_hy_2, r_rlr_2, r_rlb_2], 'ROUGE L':[r_ml_L, r_hy_L, r_rlr_L, r_rlb_L], 
              'BERTScore':[float(ml_bert_testing), float(hy_bert_testing), float(rlr_bert_testing), float(rlb_bert_testing)]}
df = pd.DataFrame(data_frame, index= ['ML', 'Hybrid', 'RL(r)', 'RL(b)']).round(4)
df

,ROUGE 1,ROUGE 2,ROUGE L,BERTScore
ML,0.3273,0.1311,0.3372,0.8372
Hybrid,0.3206,0.1250,0.3323,0.8377
RL(r),0.2803,0.1036,0.3442,0.8205
RL(b),0.3022,0.1174,0.3213,0.8395


In [146]:
!ls 'drive/My Drive/NLP PROJ/cnn_ml_model'

0011000.tar  0013000.tar  0015000.tar  0017000.tar  0019000.tar
0012000.tar  0014000.tar  0016000.tar  0018000.tar  0020000.tar


In [0]:
test_data_path = 'drive/My Drive/NLP PROJ/NLPProject/chunked/main_test/test_*'	 #"drive/My Drive/NLPProject/chunked/main_test/test_*"            #this is where you save the testing data, _* refers to multiple testing files
vocab_path = 	'drive/My Drive/NLP PROJ/NLPProject/vocab'	    #"drive/My Drive/NLPProject/vocab"                               #this is where you save the vocab file

save_model_path =  'drive/My Drive/NLP PROJ/cnn_ml_model'             #directory path for the model need to be tested


# Hyperparameters
hidden_dim = 400
emb_dim = 200
batch_size = 50
max_enc_steps = 400		
max_dec_steps = 100		
beam_size = 5
min_dec_steps= 3
vocab_size = 30000 

lr = 0.001
rand_unif_init_mag = 0.02
trunc_norm_init_std = 1e-4

eps = 1e-12
max_iterations = 10000
max_batch_queue = 100

intra_encoder = True
intra_decoder = True

In [151]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default='0015000.tar')       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()


    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_ml_1, r_ml_2, r_ml_L = eval_r.evaluate_batch()




max_size of vocab was specified as 30000; we now have 30000 words. Stopping reading.
Finished constructing vocabulary of 30000 total words. Last word added: resurrected
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
0015000.tar scores: {'rouge-1': {'f': 0.43653434823394277, 'p': 0.4021977966005998, 'r': 0.5068783269624053}, 'rouge-2': {'f': 0.20592913956989944, 'p': 0.19005306771209093, 'r': 0.23898476336358165}, 'rouge-l': {'f': 0.400196091248029, 'p': 0.3781428290719531, 'r': 0.4472944523418039}}


##WikiHow testing

In [0]:
model_folder = 'drive/My Drive/Wikihow testing'        #directory containing all testing models for Gigaword dataset

In [122]:
!ls 'drive/My Drive/Wikihow testing' 

'hybrid testing'  'ml testing'	   'RL(r) testing'
 main_test	  'RL(b) testing'   vocab




1.   **ML** model for ROUGE testing:  https://drive.google.com/file/d/115lnCmqJ6CdIqm-Oua0XR-IrEj1eDYfQ/view?usp=sharing

2.   **ML** model for BERTScore testing:  https://drive.google.com/file/d/10-vbKjRANJwrsKiHMem1kdcHqwx0j-Gu/view?usp=sharing

3.   **ML+RL** model for ROUGE testing:  https://drive.google.com/file/d/1DJ3G-aXH2BkU-60cAuCc-ErGKMUyyh8a/view?usp=sharing

4.   **ML+RL** model for BERTScore testing:  https://drive.google.com/file/d/1W7zQM9n98LlKE8J0RBLsndZlMByQlhuJ/view?usp=sharing

5.   **RL(r)** model for ROUGE testing:  https://drive.google.com/file/d/10eRbIkmAfQcGRxgUqwGFugvbFzRMCGAb/view?usp=sharing

6.   **RL(r)** model for BERTScore testing:  https://drive.google.com/file/d/1-6ph9aZiLL9wgGLbsC0Cb46zZIKoDkvI/view?usp=sharing

7.   **RL(b)** model for ROUGE testing:  https://drive.google.com/file/d/1PAY_aNuLs8Ts3whOJQpMQ5NolbG6TSrV/view?usp=sharing

8.   **RL(b)** model for BERTScore testing:  https://drive.google.com/file/d/1zWilmP2oEw0GYjpZAkAFFfAWUmw7t8Hb/view?usp=sharing

9.   **WikiHow** testing file:  https://drive.google.com/drive/folders/1oWupLbQUvmVJk4cC6hs0n4hATet0Uln2?usp=sharing

10.  **WikiHow** vocab file:  https://drive.google.com/file/d/1t6Nh8GTylnkU6naUqbGx0oVxcBe-dJie/view?usp=sharing


In [0]:
test_data_path =  (model_folder + '/main_test/test_*')	#"wiki_test/test_*"                      #testing files path
vocab_path =    (model_folder + '/vocab')		#"wiki_vocab/vocab_wiki"


# Hyperparameters
hidden_dim = 512
emb_dim = 256
batch_size = 10
max_enc_steps = 500
max_dec_steps = 100
beam_size = 4
min_dec_steps= 3
vocab_size = 20000

lr = 0.001
rand_unif_init_mag = 0.02
trunc_norm_init_std = 1e-4

eps = 1e-12
max_iterations = 16000
max_batch_queue = 30 


intra_encoder = True
intra_decoder = True

###ML testing (ROUGE+BERTScore)

In [0]:
save_model_path = (model_folder + '/ml testing')#'drive/My Drive/Gigaword/ml testing'              #directory path for the model need to be tested
model_name_for_r_testing = 'ml for ROUGE testing.tar'
model_name_for_b_testing = 'ml for BERT testing.tar'                     

In [126]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()


    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_ml_1, r_ml_2, r_ml_L = eval_r.evaluate_batch()


    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    ml_bert_testing = eval_b.evaluate_batch()

    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_ml_1, r_ml_2, r_ml_L, ml_bert_testing))

max_size of vocab was specified as 20000; we now have 20000 words. Stopping reading.
Finished constructing vocabulary of 20000 total words. Last word added: declared
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: DeprecationWarning: generator 'Batcher.text_generator' raised StopIteration


INFO:tensorflow:Finished reading dataset in single_pass mode.
ml for ROUGE testing.tar scores: {'rouge-1': {'f': 0.34325490125767216, 'p': 0.41510027642350417, 'r': 0.34732053034297433}, 'rouge-2': {'f': 0.1465648174004561, 'p': 0.17428484949760134, 'r': 0.15055950866200019}, 'rouge-l': {'f': 0.2558087953713469, 'p': 0.5565291566881132, 'r': 0.1860673566464986}}
max_size of vocab was specified as 20000; we now have 20000 words. Stopping reading.
Finished constructing vocabulary of 20000 total words. Last word added: declared
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.


KeyboardInterrupt: ignored

###Hybrid testing(ROUGE+BERTScore)

In [0]:
save_model_path = (model_folder + '/hybrid testing')        #'drive/My Drive/Gigaword/hybrid testing'              #directory path for the model need to be tested
model_name_for_r_testing = 'hybrid for ROUGE testing.tar'
model_name_for_b_testing = 'hybrid for BERT testing.tar'                

In [127]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()

    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_hy_1, r_hy_2, r_hy_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    hy_bert_testing = eval_b.evaluate_batch()
    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_hy_1, r_hy_2, r_hy_L, hy_bert_testing))

max_size of vocab was specified as 20000; we now have 20000 words. Stopping reading.
Finished constructing vocabulary of 20000 total words. Last word added: declared


KeyboardInterrupt: ignored

###RL(r) testing(ROUGE+BERTScore)

In [0]:
save_model_path =  (model_folder + '/RL(r) testing')             #'drive/My Drive/Gigaword/RL(r) testing model'              #directory path for the RL(r) model need to be tested
model_name_for_r_testing = 'RL(r) testing.tar'
model_name_for_b_testing = 'RL(r) testing.tar'                     #both names are the same for RL(r) testing

In [0]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()


    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_rlr_1, r_rlr_2, r_rlr_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    rlr_bert_testing = eval_b.evaluate_batch()

    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_rlr_1, r_rlr_2, r_rlr_L, rlr_bert_testing))

###RL(b) testing(ROUGE+BERTScore)

In [0]:
save_model_path =  (model_folder + '/RL(b) testing')         #'drive/My Drive/Gigaword/RL(b) testing model'              #directory path for the model need to be tested
model_name_for_r_testing = 'RL(b) for ROUGE testing.tar'
model_name_for_b_testing = 'RL(b) for BERT testing.tar'                  

In [0]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="test", choices=["validate","test"])
    parser.add_argument("--start_from", type=str, default=None)             
    parser.add_argument("--load_model", type=str, default=model_name_for_r_testing)       # the name of the testing model
    #opt = parser.parse_args()
    opt, unknown = parser.parse_known_args()

    eval_r = Evaluate_r(test_data_path, opt, batch_size)        #or Evaluate_b(test_data_path, opt, batch_size) if evaluated with BERTScore
    r_rlb_1, r_rlb_2, r_rlb_L = eval_r.evaluate_batch()

    opt.load_model= model_name_for_b_testing    
    eval_b = Evaluate_b(test_data_path, opt, batch_size)
    rlb_bert_testing = eval_b.evaluate_batch()
    print('ROUGE_1: {:.4f} ; ROUGE_2: {:.4f}; ROUGE_L: {:.4f}; BERTScore: {:.4f}'.format(r_rlb_1, r_rlb_2, r_rlb_L, rlb_bert_testing))

###Table

In [0]:
data_frame = {'ROUGE 1':[r_ml_1, r_hy_1, r_rlr_1, r_rlb_1], 'ROUGE 2':[r_ml_2, r_hy_2, r_rlr_2, r_rlb_2], 'ROUGE L':[r_ml_L, r_hy_L, r_rlr_L, r_rlb_L], 
              'BERTScore':[float(ml_bert_testing), float(hy_bert_testing), float(rlr_bert_testing), float(rlb_bert_testing)]}
df = pd.DataFrame(data_frame, index= ['ML', 'Hybrid', 'RL(r)', 'RL(b)']).round(4)
df